# NCSU GIS 582: Geospatial Modeling and Analysis

## 7-Optional: Surface water flow and erosion simulation

### Outline:

Here we compute surface runoff, and sediment transport using existing path sampling model. 


In [ ]:
import sys
v = sys.version_info
print(f"We are using Python {v.major}.{v.minor}.{v.micro}")

## Start GRASS GIS

Create a new mapset to run your analysis.

In [ ]:
%%bash

grass -c -e ~/grassdata/nc_spm_08_grass7/HW_7B_Flow_Modeling

In [ ]:
import subprocess
import sys

# Ask GRASS GIS where its Python packages are.
# FOR WINDOWS:
# grass_call = "grass80"
# shell_setting = True
# FOR MAC/LINUX
grass_call = "grass"
shell_setting = False

sys.path.append(
    subprocess.check_output([grass_call, "--config", "python_path"], text=True, shell=shell_setting).strip()
)

# Import GRASS packages
import grass.script as gs
import grass.jupyter as gj

# Start GRASS Session
gj.init("~/grassdata", "nc_spm_08_grass7", "HW_7B_Flow_Modeling")

## Overland flow depth and discharge

In [ ]:
%%bash
g.region rural_1m res=2 -p

Calculate partial derivatives to define the gradient of elevation surface: 

In [ ]:
%%bash
v.surf.rst -d input=elev_lid792_bepts elevation=elev_lid792_2m slope=dx_2m aspect=dy_2m tension=15 smooth=1.5 npmin=150

Note that partial derivatives can be also computed using [r.slope.aspect](https://grass.osgeo.org/grass76/manuals/r.slope.aspect.html).

Compute spatial pattern of overland flow depth and discharge by running the SIMWE model implemented in the [r.sim.water](https://grass.osgeo.org/grass76/manuals/r.sim.water.html) module. We will use uniform rainfall excess, infiltration and land cover. 

In [ ]:
%%bash
r.sim.water -t elevation=elev_lid792_2m dx=dx_2m dy=dy_2m rain_value=50 infil_value=0 man_value=0.05 depth=wdp_2m discharge=disch_2m nwalkers=100000 niterations=30 output_step=2

Display the results, add legend for individual raster and whatever you think is appropriate. 

Display Water Depth

In [ ]:
# Create Map instance
wdp_2m_30_map = gj.Map(height=600, width=600, use_region=True, filename="output/wdp_2m_30.png")
wdp_2m_30_map.d_rast(map="wdp_2m.30")
# Display map
wdp_2m_30_map.show()

Display Discharge Rate

In [ ]:
disch_2m_30_map = gj.Map(height=600, width=600, use_region=True, filename="output/disch_2m_30.png")
disch_2m_30_map.d_rast(map="disch_2m.30")
# Display map
disch_2m_30_map.show()

Create an interactive time-series map

In [ ]:
gs.run_command(
    "t.create",
    output="depth_sum",
    type="strds",
    temporaltype="absolute",
    title="Runoff Depth",
    description="Runoff Depth in [m]",
)

depth_list = gs.read_command(
    "g.list", type="raster", pattern="wdp_2m.*", separator="comma"
).strip()


gs.run_command(
    "t.register",
    input="depth_sum",
    type="raster",
    start="2023-01-01",
    increment="2 minutes",
    maps=depth_list,
    flags="i",
)

In [ ]:
# help(gj.timeseriesmap)
depth_sum_ts_map = gj.TimeSeriesMap(height=600, width=600, use_region=True)
depth_sum_ts_map.add_raster_series("depth_sum")
depth_sum_ts_map.d_legend()
depth_sum_ts_map.render()
# help(depth_sum_ts_map)
depth_sum_ts_map.save("output/depth_sum.gif")
depth_sum_ts_map.show()

![Depth](output/depth_sum.gif "Depth")

## Peak runoff with predefined flow direction

Compute peak runoff with predefined flow direction along the stream inluding culvert under the road.

First, compute direction (aspect) of the given streams: 

In [ ]:
%%bash

v.to.rast streams output=streams_dir_2m use=dir

Compute stream dx and dy using direction and slope equal to 2 degrees: 

In [ ]:
%%bash
r.mapcalc "dxstr_2m = tan(2.)*cos(streams_dir_2m)"
r.mapcalc "dystr_2m = tan(2.)*sin(streams_dir_2m)"

Compute combined DEM and stream dx and dy: 

In [ ]:
%%bash
r.mapcalc "dxdemstr_2m = if(isnull(dxstr_2m), dx_2m, dxstr_2m)"
r.mapcalc "dydemstr_2m = if(isnull(dystr_2m), dy_2m, dystr_2m)"

Run the model: 

In [ ]:
%%bash
r.sim.water -t elevation=elev_lid792_2m dx=dxdemstr_2m dy=dydemstr_2m rain_value=50 infil_value=0 man_value=0.05 depth=wdpstr_2m discharge=dischstr_2m nwalkers=100000 niterations=30 output_step=2

In [ ]:
dischstr_2m_30_map = gj.Map(height=600, width=600, use_region=True, filename="output/dischstr_2m_30.png")
dischstr_2m_30_map.d_rast(map="dischstr_2m.30")
# Display map
dischstr_2m_30_map.show()

## Runoff for spatially variable landcover and rainfall excess

Compute runoff for spatially variable landcover represented by spatially variable Mannings and rainfall excess.

We will use variable Mannings coefficient defined by reclassifying land cover class. Here is the content of a file called `land_to_mannings.txt` with reclassification rules for the  module.

```
1:1:0.9:0.9
2:2:0.5:0.5
3:3:0.01:0.01
4:4:0.03:0.03
5:5:0.01:0.01
6:6:0.05:0.05
7:7:0.1:0.1
8:8:0.1:0.1
9:9:0.9:0.9
10:10:0.03:0.03
11:11:0.5:0.5
```

In [ ]:
%%bash
r.recode input=landcover_1m output=mancover_2m rules=inputs/land_to_mannings.txt

Similarly, we will create raster with spatially variable rainfall excess rates based on the land cover classes. We use file named `land_to_rain.txt` to specify the rates for individual classes.

```
1:1:50:50
2:2:5:5
3:3:40:40
4:4:35:35
5:5:50:50
6:6:40:40
7:7:25:25
8:8:15:15
9:9:50.:50.
10:10:40:40
11:11:10:10
```

Again, we use the file as rules for the [r.recode](https://grass.osgeo.org/grass76/manuals/r.recode.html) module.

In [ ]:
%%bash
r.recode input=landcover_1m output=raincover_2m rules=inputs/land_to_rain.txt

Run the model: 

In [ ]:
%%bash
r.sim.water -t elevation=elev_lid792_2m dx=dxdemstr_2m dy=dydemstr_2m rain=raincover_2m infil_value=0 man=mancover_2m depth=wdpstrcov_2m discharge=distrcov_2m nwalkers=100000 niterations=30 output_step=2

In [ ]:
wdpstrcov_2m_30_map = gj.Map(height=600, width=600, use_region=True, filename="output/wdpstrcov_2m_30.png")
wdpstrcov_2m_30_map.d_rast(map="wdpstrcov_2m.30")
wdpstrcov_2m_30_map.d_legend(raster="wdpstrcov_2m.30")
# Display map
wdpstrcov_2m_30_map.show()

In [ ]:
distrcov_2m_30_map = gj.Map(height=600, width=600, use_region=True, filename="output/distrcov_2m_30.png")
distrcov_2m_30_map.d_rast(map="distrcov_2m.30")
distrcov_2m_30_map.d_legend(raster="distrcov_2m.30")
# Display map
distrcov_2m_30_map.show()

## Sediment flow rate, erosion and deposition

Compute sediment flow rate and net erosion/deposition using sediment transport part of the SIMWE model implemented in the the r.sim.sediment module. To make the computations faster, set region just to the upper part of the watershed: 

In [ ]:
%%bash
g.region s=s+290

Compute input transport capacity and detachment coefficient maps: 

In [ ]:
%%bash
r.mapcalc "tranin = 0.001"
r.mapcalc "detin = 0.001"

Compute input critical shear stress: 

In [ ]:
%%bash
r.mapcalc "tauin = 0.01"

Run the model using the last depth from previous run: 

In [ ]:
%%bash
g.copy rast=wdp_2m.30,wdp_2m
r.sim.sediment elevation=elev_lid792_2m dx=dx_2m dy=dy_2m water_depth=wdp_2m detachment_coeff=detin transport_coeff=tranin shear_stress=tauin man_value=0.05 nwalkers=1000000 niterations=30 transport_capacity=tcapacity tlimit_erosion_deposition=erdepmax sediment_flux=sedflow erosion_deposition=erdepsimwe

**Transport capacity raster map [kg/ms]**

In [ ]:
tcapacity_map = gj.Map(height=600, width=600, use_region=True, filename="output/tcapacity.png")
tcapacity_map.d_rast(map="tcapacity")
tcapacity_map.d_legend(raster="tcapacity")
# Display map
tcapacity_map.show()

**Transport limited erosion-deposition raster map [kg/m2s]**

In [ ]:
erdepmax_map = gj.Map(height=600, width=600, use_region=True, filename="output/erdepmax.png")
erdepmax_map.d_rast(map="erdepmax")
erdepmax_map.d_legend(raster="erdepmax")
# Display map
erdepmax_map.show()

 Display the final results: 

**Sediment flux raster map [kg/ms]**

In [ ]:
sedflow_map = gj.Map(height=600, width=600, use_region=True, filename="output/sedflow.png")
sedflow_map.d_rast(map="sedflow")
sedflow_map.d_legend(raster="sedflow")
# Display map
sedflow_map.show()

**Erosion-deposition raster map [kg/m2s]**

In [ ]:
erdepsimwe_map = gj.Map(height=600, width=600, use_region=True, filename="output/erdepsimwe.png")
erdepsimwe_map.d_rast(map="erdepsimwe")
erdepsimwe_map.d_legend(raster="erdepsimwe")
# Display map
erdepsimwe_map.show()